In [1]:
# import the necessary packages
import face_recognition
import argparse
import imutils
import pickle
import time
import cv2

In [4]:
class FaceRecognize:
    
    def __init__(self, encodings, input_file):
        # load the known faces and embeddings
        print("[INFO] loading encodings...")
        self.data = pickle.loads(open(encodings, "rb").read())
        # initialize the pointer to the video file and the video self.writer
        print("[INFO] processing video...")
        self.stream = cv2.VideoCapture(input_file)
        self.writer = None
        
    def compareEncoding(self, data, encodings):
        #print('compareEncoding(self, data, encodings):')
        names = []
        # loop over the facial embeddings
        for encoding in encodings:
            # attempt to match each face in the input image to our known
            # encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            name = "Unknown"

            # check to see if we have found a match
            if True in matches:
                # find the indexes of all matched faces then initialize a
                # dictionary to count the total number of times each face
                # was matched
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}

                # loop over the matched indexes and maintain a count for
                # each recognized face face
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number
                # of votes (note: in the event of an unlikely tie Python
                # will select first entry in the dictionary)
                name = max(counts, key=counts.get)
            # update the list of names
            names.append(name)
        return names
    
    def drawBox(self, boxes, names, frame, r):
        #print('drawBox(self, boxes, names):')
         # loop over the recognized faces
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # rescale the face coordinates
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)

            # draw the predicted face name on the image
            cv2.rectangle(frame, (left, top), (right, bottom),(0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)

    def writeVideo(self, output, frame):
        #print('writeVideo(self, output):')
        # if the video self.writer is None *AND* we are supposed to write
        # the output video to disk initialize the self.writer
        if self.writer is None and output is not None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            self.writer = cv2.VideoWriter(output, fourcc, 24, (frame.shape[1], frame.shape[0]), True)
            
        # if the self.writer is not None, write the frame with recognized
        # faces t odisk
        if self.writer is not None:
            self.writer.write(frame)
            
    def ifDisplay(self, display, frame):
        #print('ifDisplay(self, display):')
        # check to see if we are supposed to display the output frame to
        # the screen
        key = None
        if display > 0:
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(20) & 0xFF

        return key

    
    def recognizeFace(self, detection_method, output, display):
        # loop over frames from the video file stream
        while True:
            # grab the next frame
            (grabbed, frame) = self.stream.read()

            # if the frame was not grabbed, then we have reached the
            # end of the stream
            if not grabbed:
                print('breaking')
                break

            # convert the input frame from BGR to RGB then resize it to have
            # a width of 750px (to speedup processing)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb = imutils.resize(frame, width=750)
            print(rgb.shape)
            r = frame.shape[1] / float(rgb.shape[1])

            # detect the (x, y)-coordinates of the bounding boxes
            # corresponding to each face in the input frame, then compute
            # the facial embeddings for each face
            boxes = face_recognition.face_locations(rgb, model=detection_method)
            encodings = face_recognition.face_encodings(rgb, boxes)
            names = self.compareEncoding(self.data, encodings)
            self.drawBox(boxes, names, frame, r)
            self.writeVideo(output, frame) #
            key = self.ifDisplay(display, frame) # if we are supposed to display the output frame
            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
            

        # close the video file pointers
        self.stream.release()

        # check to see if the video self.writer point needs to be released
        if self.writer is not None:
            self.writer.release()

In [11]:
encodings = '/home/dai/Documents/pgdai/project/project Data/cnn_encoding.pickle'
input_file='/home/dai/Documents/pgdai/project/project Data/temp_test_data/2.mkv'
detection_method = 'cnn'
output = '/home/dai/Documents/pgdai/project/project Data/temp_output_data/out3.avi'
display=0
obj = FaceRecognize(encodings, input_file)

[INFO] loading encodings...
[INFO] processing video...


In [10]:
obj.recognizeFace(detection_method, output, display)

breaking


In [ ]:
# python recognize_faces_video_file.py --encodings encodings.pickle --input videos/lunch_scene.mp4
# python recognize_faces_video_file.py --encodings encodings.pickle --input videos/lunch_scene.mp4 --output output/lunch_scene_output.avi --display 0
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-e", "--encodings", required=True,
	help="path to serialized db of facial encodings")
ap.add_argument("-i", "--input", required=True,
	help="path to input video")
ap.add_argument("-o", "--output", type=str,
	help="path to output video")
ap.add_argument("-y", "--display", type=int, default=1,
	help="whether or not to display output frame to screen")
ap.add_argument("-d", "--detection-method", type=str, default="cnn",
	help="face detection model to use: either `hog` or `cnn`")
args = vars(ap.parse_args())